# <center> Предсказание победителя в Dota 2

### Начало

Посмотрим на готовые признаки и сделаем первую посылку. 

1. [Описание данных](#Описание-данных)
2. [Описание признаков](#Описание-признаков)
3. [Наша первая модель](#Наша-первая-модель)
4. [Посылка](#Посылка)

### Первые шаги на пути в датасайенс

5. [Кросс-валидация](#Кросс-валидация)
6. [Что есть в json файлах?](#Что-есть-в-json-файлах?)
7. [Feature engineering](#Feature-engineering)

In [ ]:
### Импорты
import os
import json
import numpy as np
import pandas as pd
import datetime
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
%matplotlib inline

In [ ]:
SEED = 10801
sns.set_style(style="whitegrid")
plt.rcParams["figure.figsize"] = 12, 8
warnings.filterwarnings("ignore")

Часть кода, который я пытался пристроить к делу, но по итогу отмел, закомментирован. Еще часть удалена совсем.

Здесь все еще хламовник, но я пытался хоть немного структурировать.

## <left>Описание данных

Файлы:

- `sample_submission.csv`: пример файла-посылки
- `train_raw_data.jsonl`, `test_raw_data.jsonl`: "сырые" данные 
- `train_data.csv`, `test_data.csv`: признаки, созданные авторами
- `train_targets.csv`: результаты тренировочных игр

## <left>Описание признаков
    
Набор простых признаков, описывающих игроков и команды в целом

In [ ]:
PATH_TO_DATA = "/kaggle/input/bi-ml-competition-2023/"

df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             "train_data.csv"), 
                                    index_col="match_id_hash")
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            "train_targets.csv"), 
                                   index_col="match_id_hash")

df_train_features.shape

df_train_features.head()

Имеем ~32 тысячи наблюдений, каждое из которых характеризуется уникальным `match_id_hash` (захэшированное id матча), и 245 признаков. `game_time` показывает момент времени, в который получены эти данные. То есть по сути это не длительность самого матча, а например, его середина, таким образом, в итоге мы сможем получить модель, которая будет предсказывать вероятность победы каждой из команд в течение матча (хорошо подходит для букмекеров).

Нас интересует поле `radiant_win` (так называется одна из команд, вторая - dire). Остальные колоки здесь по сути получены из "будущего" и есть только для тренировочных данных, поэтому на них можно просто посмотреть).

In [ ]:
df_train_targets.head()

## <left>Наша первая модель

In [ ]:
X = df_train_features.values
y = df_train_targets["radiant_win"].values.astype("int8")

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=SEED)

In [ ]:
#### Обучим случайный лес

# %%time
# rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# rf_model.fit(X_train, y_train)

#### Сделаем предсказания и оценим качество на отложенной части данных

# y_pred = rf_model.predict_proba(X_valid)[:, 1]

# valid_score = roc_auc_score(y_valid, y_pred)
# print("ROC-AUC score на отложенной части:", valid_score)

Посмотрим на accuracy:

# valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
# print("Accuracy score (p > 0.5) на отложенной части:", valid_accuracy)

## <left>Посылка

# df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, "test_data.csv"), 
#                                    index_col="match_id_hash")

# X_test = df_test_features.values
# y_test_pred = rf_model.predict_proba(X_test)[:, 1]

# df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred}, 
#                                  index=df_test_features.index)

# submission_filename = "submission_{}.csv".format(
#     datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
# df_submission.to_csv(submission_filename)
# print("Файл посылки сохранен, как: {}".format(submission_filename))

## <left>Кросс-валидация

In [ ]:
Во многих случаях кросс-валидация оказывается лучше простого разбиения на test и train. Воспользуемся `ShuffleSplit` чтобы создать 5 70%/30% наборов данных.

cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)

In [ ]:
# %%time
# rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc")

# cv_scores_rf

# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")

## <left>Что есть в json файлах?
Описание сырых данных можно найти в `train_matches.jsonl` и `test_matches.jsonl`. Каждый файл содержит одну запись для каждого матча в [JSON](https://en.wikipedia.org/wiki/JSON) формате. Его легко превратить в питоновский объект при помощи метода `json.loads`.

In [ ]:
with open(os.path.join(PATH_TO_DATA, "train_raw_data.jsonl")) as fin:
    # прочтем 419 строку
    for i in range(19):
        line = fin.readline()
    # переведем JSON в питоновский словарь 
    match = json.loads(line)

In [ ]:
# Демонстрация определния типа героя. В дальнейшем не использовалась.
for i in range(10):
    print(match["players"][i]["hero_name"])

In [ ]:
KDA - может быть неплохим признаком, этот показатель считается как:
    
<center>$KDA = \frac{K + A}{D}$

In [ ]:
player["ability_uses"]

Информация о количестве использованных способностей:

In [ ]:
sum(player["ability_uses"].values())

In [ ]:
match["players"][4]["hero_name"]

In [ ]:
for i, player in enumerate(match["players"]):
    plt.plot(player["times"], player["xp_t"], label=str(i+1))

plt.legend()
plt.xlabel("Time, s")
plt.ylabel("XP")
plt.title("XP change for all players");

In [ ]:
#### Сделаем чтение файла с сырыми данными и добавление новых признаков удобным

В этот раз для чтение `json` файлов лучше использовать библиотеку `ujson`, иначе все будет слишком долго :(

import ujson as json
from tqdm.notebook import tqdm
    
def read_matches(matches_file, total_matches=31698, n_matches_to_read=None):
    """
    Аргуент
    -------
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Возвращает записи о каждом матче
    """
    
    if n_matches_to_read is None:
        n_matches_to_read = total_matches
        
    c = 0
    with open(matches_file) as fin:
        for line in tqdm(fin, total=total_matches):
            if c >= n_matches_to_read:
                break
            else:
                c += 1
                yield json.loads(line)

In [ ]:
#### Чтение данных в цикле

Чтение всех данных занимает 1-2 минуты, поэтому для начала можно попробовать следующее:

1. Читать 10-50 игр
2. Написать код для работы с этими JSON объектами
3. Убедиться, что все работает
4. Запустить код на всем датасете
5. Сохранить результат в `pickle` файл, чтобы в следующий раз не переделывать все заново

## <left>Feature engineering

Напишем функцию, которая поможет нам легче добавлять новые признаки.

In [ ]:
def add_new_features(df_features, matches_file, n_matches=None):
    """
    Аргуенты
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
    
    for match in read_matches(matches_file, n_matches_to_read=n_matches):
        match_id_hash = match['match_id_hash']

        # Посчитаем количество разрушенных вышек обеими командами
        radiant_tower_kills = 0
        dire_tower_kills = 0
        for objective in match["objectives"]:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    radiant_tower_kills += 1
                if objective["team"] == 3:
                    dire_tower_kills += 1

        df_features.loc[match_id_hash, "radiant_tower_kills"] = radiant_tower_kills
        df_features.loc[match_id_hash, "dire_tower_kills"] = dire_tower_kills
        df_features.loc[match_id_hash, "diff_tower_kills"] = radiant_tower_kills - dire_tower_kills
        
        # ... (/¯◡ ‿ ◡)/¯☆*:・ﾟ добавляем новые признаки ...

In [ ]:
# # Скопируем таблицу с признаками
# df_train_features_extended = df_train_features.copy()

# # Добавим новые
# add_new_features(df_train_features_extended, 
#                  os.path.join(PATH_TO_DATA, 
#                               "train_raw_data.jsonl"))

# df_train_features_extended.head()

# %%time
# cv_scores_base = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc", n_jobs=-1)
# cv_scores_extended = cross_val_score(rf_model, df_train_features_extended.values, y, 
#                                      cv=cv, scoring="roc_auc", n_jobs=-1)

# print(f"ROC-AUC на кросс-валидации для базовых признаков: {cv_scores_base.mean()}")
# print(f"ROC-AUC на кросс-валидации для новых признаков: {cv_scores_extended.mean()}")

In [ ]:
### Издевательство над данными началось!

In [ ]:
df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, "test_data.csv"), 
                                   index_col="match_id_hash")

X_test = df_test_features.values
X = df_train_features
y = df_train_targets["radiant_win"].values.astype("int8")

len(df_train_features.columns)
# На старте (до джейсона) имеем 245 фичей

#### 1. Константные признаки# Их нет.

# # 1. Поищем констанстные признаки
# from sklearn.feature_selection import VarianceThreshold
# sel = VarianceThreshold(threshold=0)
# sel.fit(X)
# sum(sel.get_support())
# # Их нет.

#### 2. Квазиконстантные признакиsum(sel.get_support())


# # 1. Поищем квазиконстанстные признаки
# from sklearn.feature_selection import VarianceThreshold
# sel = VarianceThreshold(threshold=0.05)
# sel.fit(X)
# sum(sel.get_support())
# quazi_constants = [x for x in df_train_features.columns if x not in df_train_features.columns[sel.get_support()]]
# quazi_constants
# # Посмотрим внимательнее на количество убийств Рошанов, возможно, стоит удалить эти данные.

# for qc in quazi_constants:
#     print(sum(df_train_features[qc]))
# Видно, что событие достаточно редкое (встречается около 900 раз на 32000 объектов). 
# Попробуем удалить данные и сравнить скор.

# # Удаляем
# X_train_new = sel.transform(X)
# X_test_new = sel.transform(X_test)
# # Проверяем
# rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_train_new, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")
# # Кросс-валидация стала хуже

#### 3. Любуемся на признаки.

# Посмотрим на все признаки, попробуем разобраться.
# [f"Переменная -- {column}, количество уровней -- {len(set(df_train_features[column].values))}" for column in df_train_features.columns]

# set(df_train_features["lobby_type"].values)
# Лобби принимает строго 2 значения. Метаинформация и кандидат на удаление?
# У показателей XP, маны, времени оглушения, золота и здоровья большое число значений. 
# Возможно, стоит квантильно бинировать эти переменные, чтобы избавиться от шума.

# # Избавимся от лобби
# X_new = df_train_features.drop(labels=["lobby_type"], axis=1)
# print(X_new.shape)
# # Проверяем
# rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")
# # Стало Лучше на одну сотую процента. Ура, бэйзлайн перебит!

#### 4. Биннинг, недоделан

# Бининг

#### 5. Ищем и убираем скоррелированные переменные.

# corr_matrix = df_train_features.corr()
# # Делать хитмэп на такой куче данных -- гиблое дело. Поэтому будем парсить матрицу корреляции.
# corr_matrix

# # Очень кривой вариант, надо будет оптимизировать
# lab = corr_matrix.columns
# for row in lab:
#     for col in lab:
#         if corr_matrix.loc[row, col] >= 0.95 and row != col:
#             print(row, col, corr_matrix.loc[row, col])
# # Сильно скоррелированы XP и уровень (логично) и XP и золото (наверное, тоже логично, хотя корреляция прям архи-сильная)
# # Уберем XP

# # Избавимся от лобби
# X_new2 = df_train_features.drop(labels=["lobby_type", "r1_xp", "r2_xp", "r3_xp", "r4_xp", "r5_xp", "d1_xp", "d2_xp", "d3_xp", "d4_xp", "d5_xp"], axis=1)
# print(X_new.shape)
# # Проверяем
# rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")
# # Стало хуже

In [ ]:
#### 6. Команда -- больше, чем сумма игроков, ее составляющая.

In [ ]:
# Дропанье можно оптимизировать, но пока и так сойдет   (P.S. Руки так и не дошли. Это, увы, не последний кривой код в ноутбуке.)
import re
step, finish  = 50, len(df_train_features.columns)
actions = ["kills", "deaths", "assists", "denies", "gold", "lh", "xp", "health", 
           "max_health", "max_mana", "level", "stuns", "creeps_stacked", "camps_stacked",
           "rune_pickups", "firstblood_claimed", "teamfight_participation", 
           "towers_killed", "roshans_killed", "sen_placed", "obs_placed"]
commands = ["r", "d"]
X_new = df_train_features.copy()
for command in commands:
    for action in actions:
        start, stop, res = 0, 50, []
        regex = f"{command}\d_{action}"
        while stop <= finish:
            res1 = re.findall(regex, str(df_train_features.columns[start:stop]))
            if len(res1) > 0:
                res += res1
            if stop == finish:
                stop += 1
            else:
                if stop + 50 > finish:
                    stop = finish
                    start += 50
                else:
                    stop += 50
                    start += 50
        var_sum = f"{command}_{action}_sum"
        var_mean = f"{command}_{action}_mean"
        res_sum = df_train_features.filter(regex=f"{command}\d_{action}").sum(axis=1)
        res_mean = df_train_features.filter(regex=f"{command}\d_{action}").mean(axis=1)
        X_new[var_sum] = res_sum
        X_new[var_mean] = res_mean
        X_new.drop(labels=res, axis=1, inplace=True)

In [ ]:
step, finish  = 50, len(df_test_features.columns)
actions = ["kills", "deaths", "assists", "denies", "gold", "lh", "xp", "health", 
           "max_health", "max_mana", "level", "stuns", "creeps_stacked", "camps_stacked",
           "rune_pickups", "firstblood_claimed", "teamfight_participation", 
           "towers_killed", "roshans_killed", "sen_placed", "obs_placed"]
commands = ["r", "d"]
X_test_new = df_test_features.copy()
for command in commands:
    for action in actions:
        start, stop, res = 0, 50, []
        regex = f"{command}\d_{action}"
        while stop <= finish:
            res1 = re.findall(regex, str(df_test_features.columns[start:stop]))
            if len(res1) > 0:
                res += res1
            if stop == finish:
                stop += 1
            else:
                if stop + 50 > finish:
                    stop = finish
                    start += 50
                else:
                    stop += 50
                    start += 50
        var_sum = f"{command}_{action}_sum"
        var_mean = f"{command}_{action}_mean"
        res_sum = df_test_features.filter(regex=f"{command}\d_{action}").sum(axis=1)
        res_mean = df_test_features.filter(regex=f"{command}\d_{action}").mean(axis=1)
        X_test_new[var_sum] = res_sum
        X_test_new[var_mean] = res_mean
        X_test_new.drop(labels=res, axis=1, inplace=True)

In [ ]:
# Важное примечание: здесь и далее вывод скора разных моделей на кросс-валидации закомментирован, чтобы быстрее проконягться на каггле.
for_drop_mean = X_new.filter(regex=r".+mean").columns
for_drop_sum = X_new.filter(regex=r".+sum").columns
X_new_1 = X_new.drop(labels=for_drop_mean, axis=1)
X_new_2 = X_new.drop(labels=for_drop_sum, axis=1)
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_1, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")

In [ ]:
for_drop_mean = X_test_new.filter(regex=r".+mean").columns
for_drop_sum = X_test_new.filter(regex=r".+sum").columns
X_test_new_1 = X_test_new.drop(labels=for_drop_mean, axis=1)
X_test_new_2 = X_test_new.drop(labels=for_drop_sum, axis=1)

In [ ]:
##### 6.2 Поработаем с этим датасетом дальше

In [ ]:
###### 6.2.1 Уберем лобби

In [ ]:
# # Избавимся от лобби
# X_new = X_new_2.drop(labels=["lobby_type"], axis=1)
# print(X_new.shape)
# # Проверяем
# rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")
# # Стало чуть хуже

In [ ]:
###### 6.2.2 Выберем k лучших фичей

# from sklearn.feature_selection import SelectPercentile, chi2
# X_new = df_train_features.copy()
# X_new = SelectPercentile(percentile=20).fit_transform(X_new_2, y)
# X_new.shape
# # Осталось 15 фичей. Запустим лес.# Проверяем
# rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")

In [ ]:
###### 6.3 Увеличим лес и углубим деревья

for_drop_mean = X_new.filter(regex=r".+mean").columns
for_drop_sum = X_new.filter(regex=r".+sum").columns
X_new_1 = X_new.drop(labels=for_drop_mean, axis=1)
X_new_2 = X_new.drop(labels=for_drop_sum, axis=1)
rf_model = RandomForestClassifier(n_estimators=500, max_depth=10, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_1, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")
rf_model = RandomForestClassifier(n_estimators=500, max_depth=10, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")
# Скор для среднего подрос еще на процент

Поиграем еще. Будем увеличивать отдельно глубину и количество деревьев.

In [ ]:
# Наращиваем глубину
# depth = [5, 7, 10, 12, 15, 17, 20]
# for dep in depth:
#     rf_model = RandomForestClassifier(n_estimators=500, max_depth=dep, n_jobs=-1, random_state=SEED)
#     cv_scores_rf = cross_val_score(rf_model, X_new_1, y, cv=cv, scoring="roc_auc")
#     print(f"Среднее значение ROC-AUC для суммы по команде на кросс-валидации с глубиной дерева {dep}: {cv_scores_rf.mean()}")
#     rf_model = RandomForestClassifier(n_estimators=500, max_depth=dep, n_jobs=-1, random_state=SEED)
#     cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
#     print(f"Среднее значение ROC-AUC для среднего по команде на кросс-валидации с глубиной дерева {dep}: {cv_scores_rf.mean()}")

In [ ]:
На глубине 15 уже было вполне неплохо, дальше рост был меньше 3 десятых процента

# # Наращиваем количество дереьвев
# trees_number = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# for tree_number in trees_number:
#     rf_model = RandomForestClassifier(n_estimators=tree_number, max_depth=15, n_jobs=-1, random_state=SEED)
#     cv_scores_rf = cross_val_score(rf_model, X_new_1, y, cv=cv, scoring="roc_auc")
#     print(f"Среднее значение ROC-AUC для суммы по команде на кросс-валидации с {tree_number} деревьями в лесу: {cv_scores_rf.mean()}")
#     rf_model = RandomForestClassifier(n_estimators=tree_number, max_depth=15, n_jobs=-1, random_state=SEED)
#     cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
#     print(f"Среднее значение ROC-AUC для среднего по команде на кросс-валидации с {tree_number} деревьями в лесу: {cv_scores_rf.mean()}")

Выкрутим на максимум гиперпараметры и сравним с приемлемым вариантом

In [ ]:
# rf_model = RandomForestClassifier(n_estimators=400, max_depth=15, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC для среднего по команде на кросс-валидации с глубиной дерева 15 и 400 деревьями в лесу: {cv_scores_rf.mean()}")
# rf_model = RandomForestClassifier(n_estimators=900, max_depth=20, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC для среднего по команде на кросс-валидации с глубиной дерева 20 и 900 деревьями в лесу: {cv_scores_rf.mean()}")

Если мне не будет хватать двух десятых процента, чтобы кого-нибудь обогнать в таблице, я знаю, куда идти. А пока будем использовать 15, 400

In [ ]:
###### 6.4 Схлопнем KDA

X_new = X_new_2.copy()
X_new["r_kda_mean"] = (X_new_2.r_kills_mean + X_new_2.r_assists_mean)/(X_new_2.r_deaths_mean + 1)
X_new["d_kda_mean"] = (X_new_2.d_kills_mean + X_new_2.d_assists_mean)/(X_new_2.d_deaths_mean + 1)
X_new.drop(labels=["r_kills_mean", "r_assists_mean", "r_deaths_mean", 
                   "d_kills_mean", "d_assists_mean", "d_deaths_mean"], axis=1, inplace=True)
rf_model = RandomForestClassifier(n_estimators=400, max_depth=15, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC для среднего по команде на кросс-валидации с kda: {cv_scores_rf.mean()}")
# Капельку лучше

X_test_new = X_test_new_2.copy()
X_test_new["r_kda_mean"] = (X_test_new_2.r_kills_mean + X_test_new_2.r_assists_mean)/(X_test_new_2.r_deaths_mean + 1)
X_test_new["d_kda_mean"] = (X_test_new_2.d_kills_mean + X_test_new_2.d_assists_mean)/(X_test_new_2.d_deaths_mean + 1)
X_test_new.drop(labels=["r_kills_mean", "r_assists_mean", "r_deaths_mean", 
                   "d_kills_mean", "d_assists_mean", "d_deaths_mean"], axis=1, inplace=True)

In [ ]:
#### 7. Уберем координаты

# # Избавимся от лобби и координат
# X_new = df_train_features.drop(labels=["lobby_type", "r1_x", "r1_y", 
#                                       "r2_x", "r2_y", 
#                                       "r3_x", "r3_y", 
#                                       "r4_x", "r4_y", 
#                                       "r5_x", "r5_y", 
#                                       "d1_x", "d1_y", 
#                                       "d2_x", "d2_y", 
#                                       "d3_x", "d3_y", 
#                                       "d4_x", "d4_y", 
#                                       "d5_x", "d5_y", ], axis=1)
# print(X_new.shape)
# # Проверяем
# rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")
# # Стало хуже

In [ ]:
#### 8. Добавим новые фичи (сначала запустить код ниже!)

##### 8.1 Сначала базовую из ноутбука

def add_new_features(df_features, matches_file, n_matches=None):
    """
    Аргуенты
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
    
    for match in read_matches(matches_file, n_matches_to_read=n_matches):
        match_id_hash = match['match_id_hash']

        # Посчитаем количество разрушенных вышек обеими командами
        radiant_tower_kills = 0
        dire_tower_kills = 0
        for objective in match["objectives"]:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    radiant_tower_kills += 1
                if objective["team"] == 3:
                    dire_tower_kills += 1

        df_features.loc[match_id_hash, "radiant_tower_kills"] = radiant_tower_kills
        df_features.loc[match_id_hash, "dire_tower_kills"] = dire_tower_kills
        df_features.loc[match_id_hash, "diff_tower_kills"] = radiant_tower_kills - dire_tower_kills

# Скопируем таблицу с признаками
X_new_2_ext = X_new_2.copy()

# Добавим новые
add_new_features(X_new_2_ext, 
                 os.path.join(PATH_TO_DATA, 
                              "train_raw_data.jsonl"))

# Скопируем таблицу с признаками
X_test_new_2_ext = X_test_new_2.copy()

# Добавим новые
add_new_features(X_test_new_2_ext, 
                 os.path.join(PATH_TO_DATA, 
                              "test_raw_data.jsonl"), n_matches=7977)

# cv_scores_base = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc", n_jobs=-1)
# cv_scores_extended = cross_val_score(rf_model, X_new_2_ext, y, 
#                                      cv=cv, scoring="roc_auc", n_jobs=-1)
# print(f"ROC-AUC на кросс-валидации для базовых признаков: {cv_scores_base.mean()}")
# print(f"ROC-AUC на кросс-валидации для новых признаков: {cv_scores_extended.mean()}")
# На одну десятую процента лучше

In [ ]:
#### 9. Ансамбли

# import xgboost
# import lightgbm
# import catboost
# from sklearn.ensemble import (ExtraTreesClassifier,
#                               VotingClassifier)
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# # rf_model из шага с итоговым скором (дальше)
# etc = ExtraTreesClassifier(random_state=SEED)
# knn = KNeighborsClassifier(n_neighbors=5, weights="distance")
# svc_lin = SVC(kernel='linear', probability=True, random_state=SEED)
# svc_rbf = SVC(kernel='rbf', probability=True, random_state=SEED)
# cat = catboost.CatBoostClassifier(verbose=0, random_seed=SEED)
# lgbm = lightgbm.LGBMClassifier(random_state=SEED)
# lgbm_rf = lightgbm.LGBMClassifier(boosting_type="rf", bagging_freq=1, bagging_fraction=0.7, random_state=SEED)
# xgb = xgboost.XGBClassifier(random_state=SEED)
# xgb_rf = xgboost.XGBRFClassifier(random_state=SEED)
# lr = LogisticRegression(solver='liblinear', max_iter=10000)
# nb = GaussianNB()
# new_1_models = [("RF", rf_model), 
#                ("ETC", etc), ("KNN", knn), 
#                ("SVC_LIN", svc_lin), ("SVC_RBF", svc_rbf), 
#                ("CAT", cat),
#                ("LGBM_RF", lgbm_rf), ("XGB", xgb), 
#                ("XGB_RF", xgb_rf), ("LR", lr), ("NB", nb)]
# voting_hard = VotingClassifier(new_1_models, voting='hard')
# voting_soft = VotingClassifier(new_1_models, voting='soft')

# for model in [rf_model, cat, etc, knn, svc_lin, svc_rbf, xgb, lgbm, xgb_rf, lgbm_rf, lr, nb, voting_hard, voting_soft]: 
#     scores = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc", n_jobs=-1)
#     print(f"{model.__class__.__name__}: {scores.mean()}")

In [ ]:
##### 10. Работа с категориальными переменными

# [f"Переменная -- {column}, количество уровней -- {len(set(X_new_2_ext[column].values))}" for column in X_new_2_ext.columns]

# Итоговый скор

In [ ]:
rf_model = RandomForestClassifier(n_estimators=400, max_depth=15, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC для среднего по команде на кросс-валидации глубина 15 и 400 деревьев: {cv_scores_rf.mean()}")

# Много и глубоко
rf_model = RandomForestClassifier(n_estimators=900, max_depth=20, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC для среднего по команде на кросс-валидации глубина 20 и 900 деревьев: {cv_scores_rf.mean()}")

In [ ]:
# KDA
X_new_2_kda = X_new_2.copy()
X_new_2_kda["r_kda_mean"] = (X_new_2.r_kills_mean + X_new_2.r_assists_mean)/(X_new_2.r_deaths_mean + 1)
X_new_2_kda["d_kda_mean"] = (X_new_2.d_kills_mean + X_new_2.d_assists_mean)/(X_new_2.d_deaths_mean + 1)
X_new_2_kda.drop(labels=["r_kills_mean", "r_assists_mean", "r_deaths_mean", 
                   "d_kills_mean", "d_assists_mean", "d_deaths_mean"], axis=1, inplace=True)
# rf_model = RandomForestClassifier(n_estimators=900, max_depth=20, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_2_kda, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC для среднего по команде на кросс-валидации глубина 20 и 900 деревьев: {cv_scores_rf.mean()}")

X_test_new_2_kda = X_test_new_2.copy()
X_test_new_2_kda["r_kda_mean"] = (X_test_new_2.r_kills_mean + X_test_new_2.r_assists_mean)/(X_test_new_2.r_deaths_mean + 1)
X_test_new_2_kda["d_kda_mean"] = (X_test_new_2.d_kills_mean + X_test_new_2.d_assists_mean)/(X_test_new_2.d_deaths_mean + 1)
X_test_new_2_kda.drop(labels=["r_kills_mean", "r_assists_mean", "r_deaths_mean", 
                   "d_kills_mean", "d_assists_mean", "d_deaths_mean"], axis=1, inplace=True)

In [ ]:
def add_new_features(df_features, matches_file, n_matches=None):
    """
    Аргуенты
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
    
    for match in read_matches(matches_file, n_matches_to_read=n_matches):
        match_id_hash = match['match_id_hash']

        radiant_barracks_kills = 0
        dire_barracks_kills = 0         
        radiant_tower_kills = 0
        dire_tower_kills = 0
        r_aegis = 0
        d_aegis = 0
        
        for objective in match["objectives"]:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    radiant_tower_kills += 1
                if objective["team"] == 3:
                    dire_tower_kills += 1
            if objective["type"] == "CHAT_MESSAGE_AEGIS":
                if objective['player_slot'] < 100:
                    r_aegis += 1
                if objective["player_slot"] > 100:
                    d_aegis += 1

        # Посмторим, кто разрушил последнюю вышку в игре
        objs = match['objectives'][::-1]
        for objective in objs:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    last_tower_kill = 2
                if objective["team"] == 3:
                    last_tower_kill = 0
                break
            else:
                last_tower_kill = 1
                
        for i, mess in enumerate(objs):
            if mess['type'] == 'CHAT_MESSAGE_BARRACKS_KILL':
                for message in objs[i + 1:]:
                    if "slot" in message.keys():
                        if message['slot'] < 100:
                            radiant_barracks_kills += 1
                        if message['slot'] > 100:
                            dire_barracks_kills += 1
                        break

        df_features.loc[match_id_hash, "radiant_barracks_kills"] = radiant_barracks_kills
        df_features.loc[match_id_hash, "dire_barracks_kills"] = dire_barracks_kills
        df_features.loc[match_id_hash, "diff_barack_kills"] = radiant_barracks_kills - dire_barracks_kills
        df_features.loc[match_id_hash, "last_tower_kill"] = last_tower_kill
        df_features.loc[match_id_hash, "r_aegis_mean"] = r_aegis / 5
        df_features.loc[match_id_hash, "d_aegis_mean"] = d_aegis / 5
        df_features.loc[match_id_hash, "radiant_tower_kills"] = radiant_tower_kills
        df_features.loc[match_id_hash, "dire_tower_kills"] = dire_tower_kills
        df_features.loc[match_id_hash, "diff_tower_kills"] = radiant_tower_kills - dire_tower_kills

In [ ]:
# Скопируем таблицу с признаками
X_new_2_ext = X_new_2_kda.copy()

# Добавим новые
add_new_features(X_new_2_ext, 
                 os.path.join(PATH_TO_DATA, 
                              "train_raw_data.jsonl"))

In [ ]:
X_test_new_2_ext = X_test_new_2_kda.copy()

# Добавим новые
add_new_features(X_test_new_2_ext, 
                 os.path.join(PATH_TO_DATA, 
                              "test_raw_data.jsonl"), n_matches=7977)

In [ ]:
def add_new_features(df_features, matches_file, n_matches=None):
    """
    Аргуенты
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
    
    for match in read_matches(matches_file, n_matches_to_read=n_matches):
        match_id_hash = match['match_id_hash']

        # Посчитаем количество разрушенных вышек обеими командами
        r_used_abilities = 0
        d_used_abilities = 0
        r_xp_2mean = 0
        d_xp_2mean = 0
        for i in range (4):
            r_used_abilities += sum(match["players"][i]["ability_uses"].values())
            if len(match['players'][i]['xp_t']) != 0:
                r_xp_2mean += round(np.mean(match['players'][i]['xp_t']),2)
        for i in range(5,10):
            d_used_abilities += sum(match["players"][i]["ability_uses"].values())
            if len(match['players'][i]['xp_t']) != 0:
                d_xp_2mean += round(np.mean(match['players'][i]['xp_t']),2)

        df_features.loc[match_id_hash, "r_used_abilities_mean"] = r_used_abilities / 5
        df_features.loc[match_id_hash, "d_used_abilities_mean"] = d_used_abilities / 5
        df_features.loc[match_id_hash, "diff_used_abilities"] = r_used_abilities - d_used_abilities           
        df_features.loc[match_id_hash, "r_xp_2mean"] = r_xp_2mean / 5
        df_features.loc[match_id_hash, "d_xp_2mean"] = d_xp_2mean / 5

In [ ]:
X_new = X_new_2_ext.copy()

# Добавим новые
add_new_features(X_new, 
                 os.path.join(PATH_TO_DATA, 
                              "train_raw_data.jsonl"))

In [ ]:
X_test_new = X_test_new_2_ext.copy()

# Добавим новые
add_new_features(X_test_new, 
                 os.path.join(PATH_TO_DATA, 
                              "test_raw_data.jsonl"), n_matches=7977)

In [ ]:
# В удаленных ячейках были безуспешные попытки поднять скор на кросс-вале исключением золота и опыта из числа предикторов. 
corr = X_new.corr()
lab = corr.columns
for row in lab:
    for col in lab:
        if corr.loc[row, col] >= 0.95 and row != col:
            print(row, col, corr.loc[row, col])

In [ ]:
rf_model = RandomForestClassifier(n_estimators=900, max_depth=20, n_jobs=-1, random_state=SEED)
# cv_scores_rf = cross_val_score(rf_model, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")

In [ ]:
# Попробуем запустить линрегрессию
# Проверим дисбаланс классов
np.unique(y, return_counts=True)    # Все отлично

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=SEED)
# cv_scores_lr = cross_val_score(lr, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_lr.mean()}")

In [ ]:
from sklearn.ensemble import VotingClassifier
# vc_hard = VotingClassifier([("RF", rf_model), ("LR", lr)], voting='hard') # Падает
vc_soft = VotingClassifier([("RF", rf_model), ("LR", lr)],  voting='soft')
# cv_scores_vh = cross_val_score(vc_hard, X_new, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_vh.mean()}")
# cv_scores_vs = cross_val_score(vc_soft, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_vs.mean()}")

In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2
X_new_2 = X_new.copy()
X_test_new_2 = X_test_new.copy()
sp = SelectPercentile(percentile=80)
sp.fit(X_new_2, y)
X_new_trans = sp.transform(X_new_2)
X_test_new_trans = sp.transform(X_test_new_2)

In [ ]:
# vc_soft = VotingClassifier([("RF", rf_model), ("LR", lr)],  voting='soft')
# cv_scores_vs = cross_val_score(vc_soft, X_new_2, y, cv=cv, scoring="roc_auc")
# print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_vs.mean()}")

In [ ]:
# Окончательная посылка
#rf_model.fit(X_new_2, y)
vc_soft.fit(X_new_trans, y)

y_test_pred = vc_soft.predict_proba(X_test_new_trans)[:, 1]

df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred}, 
                                 index=df_test_features.index)
submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

Была еще попытка запустить LGBM на пару с оптюной, но из-за позднего старта и выдезающих отовсюду флотов, идея была отброшена.